**Sommario**

  - [`session` e `app.secret_key` in Flask](#session-e-appsecret_key-in-flask)
      - [`session`](#session)
      - [`app.secret_key`](#appsecret_key)
  - [`redirect`, `url_for` e endpoint](#redirect-url_for-e-endpoint)
      - [`redirect`](#redirect)
      - [`url_for`](#url_for)
      - [Rinominare un `endpoint`](#rinominare-un-endpoint)
    - [Sintesi](#sintesi)
  - [Uso dei "Flash" in Flask](#uso-dei-flash-in-flask)
    - [`flash`](#flash)
    - [Come funziona](#come-funziona)
    - [Sintesi](#sintesi)
  - [Uso dei template con `block` e `extends` in Flask/Jinja](#uso-dei-template-con-block-e-extends-in-flaskjinja)
      - [`block` sul template con il layout base](#block-sul-template-con-il-layout-base)
      - [Uso di `extends` e `block` sui template "specifici"](#uso-di-extends-e-block-sui-template-specifici)
    - [Come funziona](#come-funziona)
    - [Vantaggi](#vantaggi)
    - [Sintesi](#sintesi)

## `session` e `app.secret_key` in Flask

#### `session`
In Flask, la variabile `session` è un dizionario che permette di memorizzare informazioni specifiche per ogni utente, utilizzabili tra diverse richieste (requests). Utilizza un cookie lato client per conservare i dati, ma crittografa e firma questi dati per la sicurezza. Esempi di uso comune includono memorizzare lo stato di autenticazione dell'utente o altre informazioni che devono persistere tra le richieste HTTP.

#### `app.secret_key`
L'attributo `app.secret_key` è una chiave segreta che Flask utilizza per firmare i cookie di sessione. Questa chiave garantisce che i dati memorizzati nei cookie non possano essere manomessi da terze parti. Senza una chiave segreta, Flask non può garantire l'integrità e la sicurezza dei dati di sessione. È importante che questa chiave sia segreta e complessa, in modo da prevenire attacchi di tipo "brute force" e altre vulnerabilità di sicurezza.

## `redirect`, `url_for` e endpoint

#### `redirect`
La funzione `redirect` in Flask viene utilizzata per reindirizzare l'utente a un'altra pagina. È particolarmente utile dopo aver eseguito un'azione, come l'invio di un modulo, per evitare che l'utente possa inviare di nuovo i dati ricaricando la pagina.

Esempio:
```python
from flask import redirect

@app.route('/submit', methods=['POST'])
def fn_submit():
    # ...
    return redirect('/success')
```

Tuttavia, riferendosi alla route direttamente con il suo path può essere scomodo, perchè se in futuro volessimo rinominare il path a cui risponde la route, dovremmo modificare in tutto il nostro codice i riferimenti ad esso.

Per questo motivo si preferisce ottenere una route a partire dal nome della funzione che la gestisce, dato che dover rinominare una funzione capita più di rado. Per fare ciò si usa la funzione `url_for`.

#### `url_for`

La funzione `url_for` accetta il nome di un *endpoint* e restituisce un percorso assoluto per la route corrispondente. Come abbiamo detto, è particolarmente utile per evitare di scrivere in modo *hard-coded* gli URL nel codice, rendendo l'applicazione più manutenibile e meno incline a errori quando gli URL cambiano.

Esempio:
```python
from flask import redirect, url_for

@app.route('/submit', methods=['POST'])
def fn_submit():
    # ...
    return redirect(url_for('fn_success'))  # '/success'

@app.route('/success')
def fn_success():
    return "Success!"
```

In questo esempio, `url_for('fn_success')` genera l'URL per la funzione `fn_success`, ovvero `'/success'`.

#### Rinominare un `endpoint`
Un endpoint in Flask è il nome univoco assegnato a una route. Per impostazione predefinita, l'endpoint è il nome della funzione di view, ma può essere specificato esplicitamente quando si definisce la route.

Esempio:
```python
@app.route('/login', endpoint='login_page')
def login():
    return "Login Page"

url = url_for('login_page')  # '/login'
```

In questo esempio, l'endpoint per la route `/login` è esplicitamente impostato su `'login_page'`.

### Sintesi
- **`redirect()`**: reindirizza l'utente a un altro URL.
- **`url_for()`**: genera dinamicamente gli URL utilizzando il nome dell'endpoint.
- _**endpoint**_: il nome univoco assegnato a una route, utilizzato con `url_for()` per generare URL.

## Uso dei "Flash" in Flask

### `flash`
In Flask, la funzione `flash` viene utilizzata per inviare messaggi a un template, che verranno visualizzati all'utente dopo che il template viene renderizzato e inviato al client.

I messaggi flash sono particolarmente utili per notificare l'utente di eventi come errori, conferme di operazioni completate con successo, avvisi, ecc. Questi messaggi sono temporanei e vengono eliminati dopo essere stati visualizzati, al ricaricamento della pagine.

### Come funziona

1. **Impostare un messaggio Flash**
   Utilizza la funzione `flash` per impostare un messaggio.
   ```python
   from flask import flash

   @app.route('/login', methods=['POST'])
   def login():
       username = request.form['username']
       password = request.form['password']
       if username in USERS and USERS[username] == password:
           session['username'] = username
           flash('Login successful!', 'success')
           return redirect(url_for('index'))
       else:
           flash('Invalid username or password.', 'danger')
           return redirect(url_for('login'))
   ```

2. **Visualizzare i messaggi Flash nel template**
   Utilizza il seguente pattern con l'uso della funzione `get_flashed_messages` nel template per recuperare e visualizzare i messaggi flash:

   ```html
   {% with messages = get_flashed_messages(with_categories=true) %}
     {% if messages %}
       {% for category, message in messages %}
         <div class="alert alert-{{ category }}" role="alert">
           {{ message }}
         </div>
       {% endfor %}
     {% endif %}
   {% endwith %}
   ```

3. **Categorie dei messaggi Flash**
   Flask permette di configurare i messaggi flash con diverse categorie (ad esempio, `success`, `danger`, `warning`, `info`). Queste categorie possono essere utilizzate per applicare stili differenti ai messaggi, generalmente usando le classi di Bootstrap.

### Sintesi
- **`flash`**: imposta messaggi temporanei che possono essere visualizzati all'utente.
- **`get_flashed_messages`**: recupera e visualizza i messaggi flash nel template.
- **Categorie**: utilizzate per differenziare e stilizzare i messaggi (es. `success`, `danger`, `info`).

I messaggi flash migliorano l'esperienza utente fornendo un feedback immediato e chiaro sulle azioni intraprese.

## Uso dei template con `block` e `extends` in Flask/Jinja

Jinja consente di definire dei layout di base che possono essere riutilizzati modificandone solo certe parti.

#### `block` sul template con il layout base

Il comando `block` può essere utilizzato per creare un layout base che può essere esteso da altri template.

In pratica, `block` definisce delle aree che possono essere sovrascritte nei template che estendono il layout base. Ogni blocco ha un nome univoco e rappresenta una sezione del template che può essere personalizzata.

Questo consente di riutilizzare il codice e mantenere una struttura coerente in tutto il sito web.

Esempio di layout base (`base.html`):
```html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>{% block title %}Mio sito{% endblock %}</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
  <nav class="navbar navbar-expand-lg navbar-dark bg-primary">
    <a class="navbar-brand" href="#">Mio sito
  </nav>
  <div class="container mt-4">
    {% block content %}{% endblock %}
  </div>
</body>
</html>
```

#### Uso di `extends` e `block` sui template "specifici"

La combinazione dei comandi `extends` e `block` consente invece di estendere il layout base e definire i contenuti che devono essere inseriti in esso.

Ogni blocco rappresenta una sezione del template che può essere personalizzata (estesa).

Esempio di un template `home.html` che estende il template di base `base.html`:

```html
{% extends "base.html" %}

{% block title %}Home del mio sito{% endblock %}

{% block content %}
  <div class="jumbotron bg-light p-5 rounded-lg">
    <h1 class="display-4">Benvenuto nel mio sito!</h1>
    <p class="lead">Questa è la homepage del mio bellissimo sito.</p>
  </div>
{% endblock %}
```

### Come funziona

1. **Definire il layout di base**:
   - Usa `block` nel template base per definire sezioni che dovranno essere sovrascritte e nelle quali devono essere inseriti i contenuti.

2. **Estendere il layout**:
   - Nei template specifici delle pagine, utilizza `extends` indicare quale template di base deve essere ereditato ed esteso.
   - Usa `block` per definire i contenuti sepecifici che devono essere inseriti nel template di base, sovrascrivendo i blocchi in esso definiti.

### Vantaggi

- **Riutilizzo del codice**: Riduce la duplicazione del codice, facilitando la manutenzione.
- **Coerenza**: Mantiene un layout e stile uniforme in tutto il sito.
- **Facilità di aggiornamento**: Aggiornamenti al layout base si riflettono automaticamente su tutte le pagine che lo estendono.

### Sintesi

- **`{% extends 'base_template.html' %}`**: Presente nei template particolari, questa istruzione eredita il layout da un template base nominato "base_template.html".

- **`{% block title %} ... {% endblock %}`**: Nel template di base, definisce la sezione nominata "title" che può essere personalizzate nei template "particolari" che estendono il layout base. Nei template particolari, definisce il contenuto da sostituire nel template di base.

## Es. 6


- Home (benvenuto)
- Login (invio del form con credenziali di autenticazione)
    - Form (POST - FormData)
        - Campo user
        - Campo pwd
        - Bottone submit
- Film